# Lecture Notebook: Big Data and Graph Data

Apache Spark is a big data engine that runs on compute clusters, including on the cloud.  Since not everyone will have access to a compute cluster, this version of the notebook is set up to run Spark locally.


In [28]:
!apt install libkrb5-dev
!pip install sparkmagic
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libkrb5-dev is already the newest version (1.16-2ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [29]:
%load_ext sparkmagic.magics

The sparkmagic.magics extension is already loaded. To reload it, use:
  %reload_ext sparkmagic.magics


In [0]:
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    import pyspark
    from pyspark.sql import SparkSession
    from pyspark.sql.types import *
    import pyspark.sql.functions as F
    from pyspark.sql import SQLContext

In [0]:
    try:
       if(spark == None):
            spark = SparkSession.builder.appName('Graphs').getOrCreate()
            sqlContext=SQLContext(spark)
    except NameError:
        spark = SparkSession.builder.appName('Graphs').getOrCreate()
        sqlContext=SQLContext(spark)


## Example of Loading Sharded Data

First let's load the data.

In [0]:
import json
import requests

In [33]:
# If use Colab and want to mount Google Drive to it. 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [34]:
# Get 10K synthetic linked records from url X. 
#linked_in = requests.get('X')
# my_list = [json.loads(line) for line in linked_in.iter_lines()]

# Or to use that file locally. 
linked_in = open('/content/drive/My Drive/Colab Notebooks/test_data_10000.json')
my_list = [json.loads(line) for line in linked_in]

len(my_list)


10000

## Load the list into Spark

Spark needs to know the structure of the data in its dataframes, i.e., their schemas.  Since our JSON structure for LinkedIn is complex, we need to define the schema.

There are some basic types:
  * The table is a `StructType` with a list of fields (each row)
  * Most fields, in our case, are `StringType`.
  * We also have nested dictionary for the name, which is a `MapType` from `StringType` keys to `StringType` values.
  * `skills` is an `ArrayType` since it's a list, and it contains `StringType`s.
  * `also_view` is an array of structs.

See Pyspark documentation on `StructType` and examples such as https://www.programcreek.com/python/example/104715/pyspark.sql.types.StructType.

In [0]:
# Spark requires that we define a schema for the LinkedIn data...
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, MapType
schema = StructType([
        StructField("_id", StringType(), True),
        StructField("name", MapType(StringType(), StringType()), True),
        StructField("locality", StringType(), True),
        StructField("skills", ArrayType(StringType()), True),
        StructField("industry", StringType(), True),
        StructField("summary", StringType(), True),
        StructField("url", StringType(), True),
        StructField("also_view", ArrayType(\
                    StructType([\
                      StructField("url", StringType(), True),\
                      StructField("id", StringType(), True)])\
                    ), True)\
         ])

In [36]:
# Load the remote data as a list of dictionaries
linked_df = sqlContext.createDataFrame(my_list, schema).\
      repartition('_id')

linked_df.show(5)

+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _id|                name|           locality|              skills|            industry|             summary|                 url|           also_view|
+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|nvgdapwhxjjzsvycn...|[given_name -> M ...|    Uppsala, Sweden|                  []|    Ämter & Behörden|Ashley Dunning is...|jzvdtmrfybdqgeghl...|[[http://uk.linke...|
|uqerqpzcdggyrnqun...|[given_name -> Ab...|          Singapore|[{value=Motion Co...|Information Techn...|Regional Sales Ma...|cvwkibsajmjkesjvy...|[[http://in.linke...|
|wyzoingfdqnkqrvuq...|[given_name -> Ad...|Houston, Texas Area|[{value=Managemen...|Pengambilan Kakit...|I am a training a...|babrthcyozvewpzkz...|[[http:/

In [37]:
linked_df.select('name').show( truncate = False)

+------------------------------------------------------------------+
|name                                                              |
+------------------------------------------------------------------+
|[given_name -> M Burak, family_name -> Bhandari]                  |
|[given_name -> Abdelhay, family_name -> da Silva dos Santos]      |
|[given_name -> Adnan Maqbool - SAP, family_name -> Masloski]      |
|[given_name -> Anbazhagan (Anbu), family_name -> Aguinaga Azpiazu]|
|[given_name -> Steve, family_name -> Geisler]                     |
|[given_name -> Aysel, family_name -> Singer]                      |
|[given_name -> Abraham, family_name -> Asgeirsson]                |
|[given_name -> Agatha, family_name -> Choung]                     |
|[given_name -> Sean, family_name -> Ekelund]                      |
|[given_name -> Aarushi, family_name -> Åkesson]                   |
|[given_name -> Andrew A., family_name -> Baahmed]                 |
|[given_name -> Agner, family_name

In [38]:
linked_df.rdd.getNumPartitions()

200

In [39]:
linked_df.filter(linked_df.locality == 'United States')[['_id', 'name', 'locality']].show(5)

+--------------------+--------------------+-------------+
|                 _id|                name|     locality|
+--------------------+--------------------+-------------+
|dslvikjojvqncpxag...|[given_name -> Mo...|United States|
|eavnmharasuudkshz...|[given_name -> Je...|United States|
|zmjjbbtwfssfsgxnq...|[given_name -> Po...|United States|
|miagrwbtyjfnrhdvg...|[given_name -> Ag...|United States|
|pfxsthrqjqfzkvlkf...|[given_name -> Ad...|United States|
+--------------------+--------------------+-------------+
only showing top 5 rows



In [40]:
linked_df.select("_id", "locality").show(5)

+--------------------+-------------------+
|                 _id|           locality|
+--------------------+-------------------+
|nvgdapwhxjjzsvycn...|    Uppsala, Sweden|
|uqerqpzcdggyrnqun...|          Singapore|
|wyzoingfdqnkqrvuq...|Houston, Texas Area|
|yitfbojzyhfmhdwdx...| Paris Area, France|
|gliizzsriqmgyfqop...|            Türkiye|
+--------------------+-------------------+
only showing top 5 rows



In [0]:
### Clean out the list from memory
my_list = []

In [42]:
linked_df.createOrReplaceTempView('linked_in')
sqlContext.sql('select * from linked_in').show(5)

+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _id|                name|           locality|              skills|            industry|             summary|                 url|           also_view|
+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|nvgdapwhxjjzsvycn...|[given_name -> M ...|    Uppsala, Sweden|                  []|    Ämter & Behörden|Ashley Dunning is...|jzvdtmrfybdqgeghl...|[[http://uk.linke...|
|uqerqpzcdggyrnqun...|[given_name -> Ab...|          Singapore|[{value=Motion Co...|Information Techn...|Regional Sales Ma...|cvwkibsajmjkesjvy...|[[http://in.linke...|
|wyzoingfdqnkqrvuq...|[given_name -> Ad...|Houston, Texas Area|[{value=Managemen...|Pengambilan Kakit...|I am a training a...|babrthcyozvewpzkz...|[[http:/

In [43]:
sqlContext.sql('select _id, name.given_name, name.family_name from linked_in').show(10)

+--------------------+-------------------+-------------------+
|                 _id|         given_name|        family_name|
+--------------------+-------------------+-------------------+
|nvgdapwhxjjzsvycn...|            M Burak|           Bhandari|
|uqerqpzcdggyrnqun...|           Abdelhay|da Silva dos Santos|
|wyzoingfdqnkqrvuq...|Adnan Maqbool - SAP|           Masloski|
|yitfbojzyhfmhdwdx...|  Anbazhagan (Anbu)|   Aguinaga Azpiazu|
|gliizzsriqmgyfqop...|              Steve|            Geisler|
|nilkycgqibqzhcpws...|              Aysel|             Singer|
|cbeqtauvkrpwjdxno...|            Abraham|         Asgeirsson|
|aamexaalrxcxvcvmt...|             Agatha|             Choung|
|ojpxbwcehftbsicle...|               Sean|            Ekelund|
|bpfnuxzybuusoiubr...|            Aarushi|            Åkesson|
+--------------------+-------------------+-------------------+
only showing top 10 rows



In [44]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

acro = udf(lambda x: ''.join([n[0] for n in x.split()]), StringType())

linked_df.select("_id", acro("locality").alias("acronym")).show(5)

+--------------------+-------+
|                 _id|acronym|
+--------------------+-------+
|nvgdapwhxjjzsvycn...|     US|
|uqerqpzcdggyrnqun...|      S|
|wyzoingfdqnkqrvuq...|    HTA|
|yitfbojzyhfmhdwdx...|    PAF|
|gliizzsriqmgyfqop...|      T|
+--------------------+-------+
only showing top 5 rows



In [45]:
# Which industries are most popular?
sqlContext.sql('select count(_id), industry '+\
               'from linked_in '+\
               'group by industry '+\
               'order by count(_id) desc').\
    show(5)

+----------+--------------------+
|count(_id)|            industry|
+----------+--------------------+
|       323|Information Techn...|
|       212|   Computer Software|
|       112|            Internet|
|       102|Marketing and Adv...|
|        72|  Financial Services|
+----------+--------------------+
only showing top 5 rows



## Graphs

For the next set of examples, we will look at graph-structured data.  It turns out our LinkedIn dataset has a list of nodes (by int ID, but associated with the user ID we used in the linked_in table) and a list of edges.

In [0]:
import urllib
import zipfile
import os

# Replace the URL with the correct address of the edges.zip file
url = 'https://xxx/opends4all/linkedin.edges.zip'
filehandle, _ = urllib.request.urlretrieve(url)

zip_file_object = zipfile.ZipFile(filehandle, 'r')
fname = zip_file_object.open('linkedin.edges')

edges = []
MAX = 2000000

for link in fname:
  edge = link.decode('utf-8').split(' ')
  edges.append([int(edge[0]), int(edge[1])])
  if len(edges) >= MAX:
    break


In [88]:
from pyspark.sql.types import IntegerType
schema = StructType([
        StructField("from", IntegerType(), True),
        StructField("to", IntegerType(), True)
         ])
# Load the remote data as a list of dictionaries
edges_df = sqlContext.createDataFrame(edges, schema)

edges_df.show(5)

+-------+-------+
|   from|     to|
+-------+-------+
|2282701|3912501|
|2282701|5182389|
|2282701|3822131|
|2282701|8034158|
|2282701|1946731|
+-------+-------+
only showing top 5 rows



In [89]:
edges_df.createOrReplaceTempView('edges')
sqlContext.sql('select from as id, count(to) as degree from edges group by from').show(5)

+-------+------+
|     id|degree|
+-------+------+
|7221550|   148|
|6476312|   130|
|4200201|   129|
|6494598|   106|
|1223588|    89|
+-------+------+
only showing top 5 rows



## Traversing the Graph

In [90]:
from pyspark.sql.functions import col

# Start with a subset of nodes
start_nodes_df = edges_df[['from']].filter(edges_df['from'] < 100000).\
  select(col('from').alias('id')).drop_duplicates()

start_nodes_df.show(5)

# The neighbors require us to join
# and we'll use Spark DataFrames syntax here
neighbor_nodes_df = start_nodes_df.\
  join(edges_df, start_nodes_df.id == edges_df['from']).\
  select(col('to').alias('id'))

neighbor_nodes_df.show(5)

+-----+
|   id|
+-----+
|71510|
|74058|
|76756|
|  148|
|50353|
+-----+
only showing top 5 rows

+-------+
|     id|
+-------+
|9777199|
|1743876|
|2207408|
|7836663|
|9431178|
+-------+
only showing top 5 rows



In [91]:
edges_df[['from']].orderBy('from').drop_duplicates().show()

edges_df.filter(edges_df['from'] == 698).show()

+----+
|from|
+----+
| 123|
| 148|
| 322|
| 698|
|1077|
|1144|
|1162|
|1302|
|1393|
|1504|
|1525|
|1638|
|1678|
|1808|
|1939|
|2163|
|2514|
|2582|
|2714|
|2952|
+----+
only showing top 20 rows

+----+-------+
|from|     to|
+----+-------+
| 698|4067826|
| 698| 565155|
| 698|1009367|
| 698|7264684|
| 698|8609691|
| 698|8797222|
| 698|2078369|
| 698|9390344|
| 698|2343229|
| 698|4874249|
| 698|2545215|
| 698|3127567|
| 698|4760779|
| 698|4460232|
| 698|4674327|
| 698|7848239|
| 698|2324983|
| 698|9215385|
| 698|5698971|
| 698|5996191|
+----+-------+
only showing top 20 rows



In [92]:
neighbor_neighbor_nodes_df = neighbor_nodes_df.\
  join(edges_df, neighbor_nodes_df.id == edges_df['from']).\
  select(col('to').alias('id'))

neighbor_neighbor_nodes_df.show(5)

+-------+
|     id|
+-------+
|6035498|
|6166961|
|2448237|
|1905292|
|1747364|
+-------+
only showing top 5 rows



In [0]:
def iterate(df, depth):
  df.createOrReplaceTempView('iter')

  # Base case: direct connection
  result = sqlContext.sql('select from, to, 1 as depth from iter')

  for i in range(1, depth):
    result.createOrReplaceTempView('result')
    result = sqlContext.sql('select r1.from as from, r2.to as to, r1.depth+1 as depth  '\
                            'from result r1 join iter r2 '\
                            'on r1.to=r2.from')
  return result

In [98]:
iterate(edges_df.filter(edges_df['from'] < 1000000), 1).orderBy('from','to').show()

+----+-------+-----+
|from|     to|depth|
+----+-------+-----+
| 123|  91543|    1|
| 123| 766201|    1|
| 123| 805244|    1|
| 123|1115889|    1|
| 123|1460837|    1|
| 123|1837525|    1|
| 123|2397963|    1|
| 123|3117349|    1|
| 123|3499006|    1|
| 123|4090527|    1|
| 123|4937958|    1|
| 123|5057050|    1|
| 123|6277751|    1|
| 123|7789741|    1|
| 123|7956667|    1|
| 123|8022868|    1|
| 123|8212249|    1|
| 123|8688764|    1|
| 123|9166545|    1|
| 123|9283427|    1|
+----+-------+-----+
only showing top 20 rows



In [99]:
iterate(edges_df.filter(edges_df['from'] < 1000000), 2).orderBy('from','to').show()

+----+-------+-----+
|from|     to|depth|
+----+-------+-----+
|1678| 868412|    2|
|1678|1855400|    2|
|1678|2762625|    2|
|1678|3558053|    2|
|1678|4929260|    2|
|1678|5112663|    2|
|1678|5459172|    2|
|1678|6132561|    2|
|1678|6314741|    2|
|1678|6444050|    2|
|1678|7038299|    2|
|1678|7394947|    2|
|1678|7802268|    2|
|1678|8364779|    2|
|1678|8504601|    2|
|1678|9401691|    2|
|1678|9416271|    2|
|1678|9993870|    2|
|2582| 288213|    2|
|2582| 504980|    2|
+----+-------+-----+
only showing top 20 rows



In [100]:
iterate(edges_df.filter(edges_df['from'] < 1000000), 3).orderBy('from','to').show()

+-----+-------+-----+
| from|     to|depth|
+-----+-------+-----+
|15305| 859468|    3|
|15305| 919212|    3|
|15305|1032892|    3|
|15305|1079033|    3|
|15305|1346621|    3|
|15305|2324259|    3|
|15305|2451906|    3|
|15305|3249045|    3|
|15305|3696999|    3|
|15305|5032700|    3|
|15305|5197855|    3|
|15305|5447389|    3|
|15305|6836284|    3|
|15305|7912140|    3|
|15305|8331813|    3|
|15305|9103098|    3|
|15305|9826214|    3|
|16247| 731360|    3|
|16247| 733043|    3|
|16247|1893329|    3|
+-----+-------+-----+
only showing top 20 rows



In [0]:
# Clear list of edges from Python memory
# to free up space
edges = []


### Now let's get the list of node IDs
url='https://xxx/opends4all/linkedin.nodes.zip'
nodehandle, _ = urllib.request.urlretrieve(url)

zip_file_object = zipfile.ZipFile(nodehandle, 'r')
fname = zip_file_object.open('linkedin.nodes')

nodes = []
MAX = 100000

for node in fname:
  node_tuple = node.decode('utf-8').split()
  nodes.append([int(node_tuple[0]), str(node_tuple[1])])
  if len(nodes) >= MAX:
    break


## Joins in Spark, Beyond Graph Traversals

What if we want to connect our edges to the people from our previous crawl?  Sadly the edges use int node IDs that don't correspond to the people dataframe.  But in fact the node data includes this information, so let's load and exploit that.

Let's load the information about nodes, and their correspondence to the user ID.

In [102]:
schema = StructType([
        StructField("nid", IntegerType(), True),
        StructField("user", StringType(), True)
         ])
# Load the remote data as a list of dictionaries
nodes_df = sqlContext.createDataFrame(nodes, schema)

nodes_df.show(5)

+-------+-------------+
|    nid|         user|
+-------+-------------+
|2282701|newin_2282701|
|9582258|newin_9582258|
| 853296| newin_853296|
| 869746| newin_869746|
|6899568|newin_6899568|
+-------+-------------+
only showing top 5 rows



## Finding Friends, by ID

In [103]:
nodes_df.createOrReplaceTempView('nodes')
edges_df.createOrReplaceTempView('edges')

friends_df = \
sqlContext.sql('select n1.user, n2.user as friend ' +\
               'from (nodes n1 join edges e on n1.nid = e.from) join nodes n2 on e.to = n2.nid')

friends_df.show(5)


+-------------+-----------+
|         user|     friend|
+-------------+-----------+
|newin_8425340|  newin_148|
|newin_8423898|newin_57370|
| newin_942965|newin_76756|
| newin_215843|newin_76756|
|newin_4674072|newin_76756|
+-------------+-----------+
only showing top 5 rows



## Connecting Friends to Names

In [104]:
friends_df.createOrReplaceTempView('friends')

sqlContext.sql('select u1.name.given_name as user, u2.name.given_name as friend '+\
               'from (linked_in u1 join friends on u1._id = user) join linked_in u2 on u2._id = friend').show(5)

+----+------+
|user|friend|
+----+------+
|Adam|  Abiy|
|Abhi|Adukwu|
+----+------+

